### Importing libraries

In [10]:
from models import Yolov4
model = Yolov4(weight_path='yolov4.weights',
               class_name_path='class_names/coco_classes.txt')
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import numpy as np
import easyocr
from PIL import Image, ImageDraw

nms iou: 0.413 score: 0.3
all weights read
load from yolov4.weights


### Loading dataframe from CSV

In [155]:
features = ['time','frame_name','frame_name_2','lat','lon','hmsl','ggs','gc','hacc','satellite','ping_edge','ping_six','iperf_down','iperf_up','netmode']


df = pd.read_csv('data/2023-01-05_10_56_51.985284_merged.csv', skiprows=2390, nrows=10, header=None, names=features)

In [156]:
print(df.shape)
df.head()

(10, 15)


,time,frame_name,frame_name_2,lat,lon,hmsl,ggs,gc,hacc,satellite,ping_edge,ping_six,iperf_down,iperf_up,netmode
0,2023-01-05 10:58:11.427480,frame_1_2390.jpg,NaN,48.154337,17.074862,159165.9,724,128.64888,14.1,44,26.948,37.741,146532.0,23957.0,5G-NR
1,2023-01-05 10:58:11.462298,frame_1_2391.jpg,NaN,48.154337,17.074862,159165.9,724,128.64888,14.1,44,26.948,37.741,146532.0,23957.0,5G-NR
2,2023-01-05 10:58:11.493051,frame_1_2392.jpg,NaN,48.154337,17.074862,159165.9,724,128.64888,14.1,44,26.948,37.741,146532.0,23957.0,5G-NR
3,2023-01-05 10:58:11.526576,frame_1_2393.jpg,NaN,48.154337,17.074862,159165.9,724,128.64888,14.1,44,26.948,37.741,146532.0,23957.0,5G-NR
4,2023-01-05 10:58:11.558693,frame_1_2394.jpg,NaN,48.154337,17.074862,159165.9,724,128.64888,14.1,44,26.948,37.741,146532.0,23957.0,5G-NR


### Image preprocessing functions

In [163]:
def increase_contrast(image, alpha=1, beta=100):
    result = np.zeros(image.shape, dtype=np.uint8)
    
    cv2.convertScaleAbs(image, result, alpha=alpha, beta=beta)
    
    return result


def preprocess_image(image):
    #norm_img = np.zeros((image.shape[0], image.shape[1]))
    # Image normalization
    #image = cv2.normalize(image, norm_img, 0, 255, cv2.NORM_MINMAX)
    # Increasing contrast for better readability
    # Removing noise from color image
    #image = cv2.fastNlMeansDenoisingColored(image, None, 2, 2, 5, 3)
    #image = cv2.resize(image, (600, 400))
    
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    image = increase_contrast(gray_image)
    
    # Image segmentation
    #blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)
    #_, result = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    return image

def is_negative(num):
    return num if num >= 0 else 0

### Blurring license plates
- Detection of cars in the image
- The use of the model to recognize license plates 
- The use of easyocr for text detection in the image

In [165]:
reader = easyocr.Reader(['en'])
model1 = cv2.CascadeClassifier('haarcascade_russian_plate_number.xml')

# Iteration through the dataframe
for index, row in df.iterrows():
    
    file_name = 'img/' + row['frame_name']
    print(file_name)
    
    # Object prediction with YoLo 
    predictions = model.predict(file_name)
    # Filtering only vehicles
    predictions = predictions.loc[(predictions['class_name'].isin(['car', 'bus'])) & (predictions['w'] > 50) & (predictions['h'] > 50)]

    image = cv2.imread(file_name)
    
    # Iterate over individual vehicles
    for idx, car in predictions.iterrows():
        print(file_name + str(idx))
        # Cutting out the vehicle from the image
        c_x, c_y, c_h, c_w = car['x1'], car['y1'], car['h'], car['w']
        cropped_image = image[c_y:c_y + c_h, c_x:c_x + c_w] 
        
        # The use of a model to recognize license plates
        plates = model1.detectMultiScale(cropped_image, scaleFactor=1.01, minNeighbors=7, maxSize=(100,100))
        
        if(len(plates) != 0):
            print(file_name + str(idx) + ' Model')
            
        for (x, y, w, h) in plates:
            car_roi = cropped_image[y:y+h, x:x+w]
            blur = cv2.GaussianBlur(car_roi, (25, 25), 0)
            cropped_image[y:y+h, x:x+w] = blur

            image[c_y:c_y + c_h, c_x:c_x + c_w] = cropped_image

        # Car image preprocessing
        preprocessed_image = preprocess_image(cropped_image)
        
        # Using easyocr to search for text on car image
        #plt.imshow(preprocessed_image, cmap='gray')
        #plt.show()
        
        results = reader.readtext(preprocessed_image, detail=1, paragraph=False)
        if(results):
            for result in results:
                print(file_name + str(idx) + ' easyocr')
                crd = result[0]
                t_x1, t_y1 = int(is_negative(crd[0][0])), int(is_negative(crd[0][1]))
                t_x2, t_y2 = int(is_negative(crd[2][0])), int(is_negative(crd[2][1]))

                t_w = t_x2 - t_x1
                t_h = t_y2 - t_y1

                main_text_x = c_x + t_x1
                main_text_y = c_y + t_y1

                blur = cv2.GaussianBlur(image[main_text_y:main_text_y + t_h, main_text_x:main_text_x+t_w], (15, 15), 30)
                image[main_text_y:main_text_y + t_h, main_text_x:main_text_x + t_w] = blur 

    cv2.imwrite('new/' + row['frame_name'], image)


CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


img/frame_1_2390.jpg
1/1 [==============================] - 1s 528ms/step
# of bboxes: 13
img/frame_1_2390.jpg0
img/frame_1_2391.jpg
1/1 [==============================] - 1s 619ms/step
# of bboxes: 13
img/frame_1_2391.jpg0
img/frame_1_2392.jpg
1/1 [==============================] - 1s 760ms/step
# of bboxes: 11
img/frame_1_2392.jpg0
img/frame_1_2393.jpg
1/1 [==============================] - 1s 545ms/step
# of bboxes: 10
img/frame_1_2393.jpg2
img/frame_1_2394.jpg
1/1 [==============================] - 1s 557ms/step
# of bboxes: 11
img/frame_1_2394.jpg0
img/frame_1_2395.jpg
1/1 [==============================] - 1s 548ms/step
# of bboxes: 9
img/frame_1_2395.jpg0
img/frame_1_2396.jpg
1/1 [==============================] - 1s 727ms/step
# of bboxes: 9
img/frame_1_2396.jpg0
img/frame_1_2397.jpg
1/1 [==============================] - 1s 559ms/step
# of bboxes: 11
img/frame_1_2397.jpg0
img/frame_1_2398.jpg
1/1 [==============================] - 1s 584ms/step
# of bboxes: 10
img/frame_1_2398